In [138]:
# import our dependencies

%matplotlib inline
from collections import Counter
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import numpy as np

In [139]:
# import our file

Beverly_Grove_df = pd.read_csv('Beverly_Grove_Cleaned.csv', index_col='PARCEL')
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
Beverly_Grove_df = Beverly_Grove_df.dropna()

In [141]:
Beverly_Grove_df.dtypes

Owned by Trust?                int64
Owned by Business?             int64
SITEADDRESS                   object
MAIL DIFFERENT FROM SITE?      int64
MAIL OUTSIDE CA?               int64
TITLECO1                       int64
ASSDTOTAL                      int64
ASSDLAND                       int64
ASSDSTCT                       int64
ASSDOTHR                       int64
EXEMPTCD                       int64
EXMPTAMT                       int64
PCNTIMPD                       int64
TAXAMT                       float64
DOCDATE_YEAR                   int64
MULTIPARCEL                    int64
PRICE                          int64
LENDER1                        int64
LOANAMOUT1                     int64
LOANTYPE1                      int64
YEARBLT                        int64
EFFYRBLT                     float64
LOTSQFT                        int64
TOTALSF                        int64
GRGTYPE                        int64
BLDGMAT                        int64
TOTUNITS                       int64
Q

In [142]:
Beverly_Grove_df['TAXAMT'] = Beverly_Grove_df['TAXAMT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(int)

In [143]:
# Make sure our dataframe is only SFR's so exclude every property that has an '#' in its site address

Beverly_Grove_df = Beverly_Grove_df[~Beverly_Grove_df['SITEADDRESS'].str.contains('#')]
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
# Now drop the SiteAddress column
Beverly_Grove_df.drop(['SITEADDRESS'], axis=1, inplace=True)

In [145]:
# check to see if it dropped.
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0


In [146]:
len(Beverly_Grove_df)

4705

# Attempt 1: Run a standard Logistic Regression

In [147]:
# Step 1: Create our X and y

y = Beverly_Grove_df['Did it sell?']
X = Beverly_Grove_df.drop(['Did it sell?'], 1)

In [148]:
APN = Beverly_Grove_df.index.values
APN

array([5085010022, 5085033008, 5085032022, ..., 5510018012, 5510017023,
       5514001029])

In [149]:
# Split our data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

In [150]:
X_train.head(3)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5088017017,0,0,0,0,1,340000,136000,204000,0,1,...,0,0,0,0,0,0,0,0,0,0
5089020010,0,0,0,0,0,218091,106556,111535,0,1,...,0,0,0,0,0,0,0,0,0,0
5088016028,1,0,0,0,1,195815,49267,146548,0,1,...,0,0,0,0,0,0,0,0,0,0


In [151]:
X_train.columns

Index(['Owned by Trust?', 'Owned by Business?', 'MAIL DIFFERENT FROM SITE?',
       'MAIL OUTSIDE CA?', 'TITLECO1', 'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT',
       'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT', 'PCNTIMPD', 'TAXAMT',
       'DOCDATE_YEAR', 'MULTIPARCEL', 'PRICE', 'LENDER1', 'LOANAMOUT1',
       'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF', 'GRGTYPE',
       'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY', 'ROOFMAT',
       'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM', 'POOL',
       'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'VIEW', 'GRANT DEED',
       'IN LIEU OF FORC', 'INTERSPOUSAL', 'JOINT TENANT', 'NAME CHANGE',
       'Other', 'PARTNERSHIP', 'PERSONAL REP', 'QUIT CLAIM', 'RE-RECORD',
       'TAX DEED', 'TRUST TRANSFER', 'TRUSTEES', 'WARRANTY'],
      dtype='object')

In [152]:
# Scale our data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit our scaler, named 'scaler' to our data, which produces a new StandardScaler object
# which we call 'X_scaler'
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [153]:
X_train_scaled_APN = []

In [154]:
for index, parcel in enumerate(X_train.index.values):
     X_train_scaled_APN.append(np.insert(X_train_scaled[index], 0, [parcel]))

In [155]:
# Create the learning model

classifier = LogisticRegression(solver='lbfgs',
                                max_iter=300,
                                random_state=78,
                                class_weight="balanced")

In [156]:
# fit our learning model to our scaled training data

classifier.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [157]:
y_pred = classifier.predict(X_test_scaled)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test, "PARCEL": y_test.index.values}).reset_index(drop=True)
results.sample(20)

,Prediction,Actual,PARCEL
225,1,0,5086017028
974,0,0,5526037002
207,0,0,5510017030
1036,0,0,5085020014
194,1,0,5510003015
290,1,0,4332021012
427,0,0,5088017001
418,1,0,5514001025
851,0,0,5089020007
739,0,1,5085022013


In [158]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.5921835174171622


In [159]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average="weighted")

0.6923828010441901

In [160]:
# See our results in a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[666, 430],
       [ 50,  31]])

In [161]:
# evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.61      0.74      1096
           1       0.07      0.38      0.11        81

    accuracy                           0.59      1177
   macro avg       0.50      0.50      0.42      1177
weighted avg       0.87      0.59      0.69      1177



In [162]:
Beverly_Grove_df = Beverly_Grove_df.merge(results, how="left", right_on="PARCEL", left_on="PARCEL")
Beverly_Grove_df.head()

,PARCEL,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY,Prediction,Actual
0,5085010022,0,0,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,5085033008,0,0,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0.0,0.0
2,5085032022,0,0,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,5510005006,1,0,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0.0,0.0
4,5089024002,0,0,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,NaN,NaN


# Attempt 2: Resample our data using SMOTEEN

In [163]:
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({0: 2095, 1: 3245})

In [164]:
model_2 = LogisticRegression(solver='lbfgs', random_state=78, max_iter=300, class_weight="balanced")
model_2.fit(X_resampled, y_resampled)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [165]:
# Calculated the balanced accuracy score
y_pred = model_2.predict(X_test_scaled)

In [166]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5283635216725241

In [167]:
f1_score(y_test, y_pred, average="weighted")

0.6818244603198075

In [168]:
confusion_matrix(y_test, y_pred)

array([[644, 452],
       [ 43,  38]])

In [169]:
# evaluate
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.59      0.72      1096
           1       0.08      0.47      0.13        81

    accuracy                           0.58      1177
   macro avg       0.51      0.53      0.43      1177
weighted avg       0.88      0.58      0.68      1177



# Attempt 3: Resample our Data using SMOTE

In [170]:
from imblearn.over_sampling import SMOTE
X_resampled_2, y_resampled_2 = SMOTE(random_state=1, 
                               sampling_strategy='auto').fit_resample(
                               X_train_scaled, y_train)

In [171]:
Counter(y_resampled_2)

Counter({0: 3287, 1: 3287})

In [172]:
# perform our logistic regression

model_3 = LogisticRegression(solver='lbfgs', random_state=78, max_iter=300, class_weight="balanced")
model_3.fit(X_resampled_2, y_resampled_2)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [173]:
# Calculated the balanced accuracy score
y_pred = model_3.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.5146492295214923

In [174]:
f1_score(y_test, y_pred, average="weighted")

0.6785693826192267

# Attempt 4: Use a Support Vector Machine

In [175]:
# Instantiate a linear SVM model
from sklearn.svm import SVC
model_4 = SVC(kernel='linear')

In [176]:
#model_4.class_weight_

In [177]:
#model_4.coef_

In [178]:
#pd.Series(abs(model_4.coef_[0]), index=X_train.columns).nlargest(20).plot(kind='barh')

In [179]:
Beverly_Grove_df.columns

Index(['PARCEL', 'Owned by Trust?', 'Owned by Business?',
       'MAIL DIFFERENT FROM SITE?', 'MAIL OUTSIDE CA?', 'TITLECO1',
       'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT', 'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT',
       'PCNTIMPD', 'TAXAMT', 'DOCDATE_YEAR', 'MULTIPARCEL', 'PRICE', 'LENDER1',
       'LOANAMOUT1', 'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF',
       'GRGTYPE', 'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY',
       'ROOFMAT', 'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM',
       'POOL', 'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'VIEW',
       'Did it sell?', 'GRANT DEED', 'IN LIEU OF FORC', 'INTERSPOUSAL',
       'JOINT TENANT', 'NAME CHANGE', 'Other', 'PARTNERSHIP', 'PERSONAL REP',
       'QUIT CLAIM', 'RE-RECORD', 'TAX DEED', 'TRUST TRANSFER', 'TRUSTEES',
       'WARRANTY', 'Prediction', 'Actual'],
      dtype='object')

In [180]:
#model_4.support_vectors_

In [181]:
# Fit/train our model using the scaled training data
model_4.fit(X_train_scaled, y_train)

SVC(kernel='linear')

In [182]:
# Make predictions using the test data
y_pred = model_4.predict(X_test_scaled)
results_SVM = pd.DataFrame({
    "Prediction": y_pred,
    "Actual": y_test
}).reset_index(drop=True)

In [183]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9311809685641461

In [184]:
balanced_accuracy_score(y_test, y_pred)

0.5

In [185]:
f1_score(y_test, y_pred, average="weighted")

0.8979976608414467

In [186]:
# create our confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1096,    0],
       [  81,    0]])

In [187]:
# evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1096
           1       0.00      0.00      0.00        81

    accuracy                           0.93      1177
   macro avg       0.47      0.50      0.48      1177
weighted avg       0.87      0.93      0.90      1177



/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

## Attempt 5: Reduce the number of columns to just the important ones and run an SVC

In [188]:
# REimport our file

# Beverly_Grove_df = pd.read_csv('Beverly_Grove_Cleaned.csv', index_col='PARCEL')
# Beverly_Grove_df.head()

In [189]:
Beverly_Grove_Reduced_df = Beverly_Grove_df[['MAIL DIFFERENT FROM SITE?',
                                             'ASSDTOTAL',
                                             'ASSDLAND',
                                             'ASSDSTCT',
                                             'BLDGCOND',
                                             'TAXAMT',
                                             'PRICE',
                                             'YEARBLT',
                                             'EFFYRBLT',
                                             'LOANAMOUT1',
                                             'Did it sell?']]

In [190]:
#Beverly_Grove_Reduced_df.set_index(['PARCEL'], inplace=True)

In [191]:
Beverly_Grove_Reduced_df.head(3)

,MAIL DIFFERENT FROM SITE?,ASSDTOTAL,ASSDLAND,ASSDSTCT,BLDGCOND,TAXAMT,PRICE,YEARBLT,EFFYRBLT,LOANAMOUT1,Did it sell?
0,0,1157385,925908,231477,1,14144,1140000,1905,1938,775200,0
1,1,57047,40994,16053,0,858,0,1906,1916,0,0
2,0,651485,521193,130292,0,8044,0,1909,1937,0,0


In [192]:
len(Beverly_Grove_Reduced_df)

4705

In [193]:
# create new X and y values based on the Beverly_Grove_Reduced dataframe

y_2 = Beverly_Grove_Reduced_df['Did it sell?']
X_2 = Beverly_Grove_Reduced_df[['MAIL DIFFERENT FROM SITE?',
                              'ASSDTOTAL',
                              'ASSDLAND',
                              'ASSDSTCT',
                              'BLDGCOND',
                              'TAXAMT',
                              'PRICE',
                              'YEARBLT',
                              'EFFYRBLT',
                              'LOANAMOUT1']]

In [194]:
# create new training and testing data

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, random_state=78, stratify=y)

In [195]:
# Scale our new training and test data

# Create the StandardScaler instnce
scaler_2 = StandardScaler()

In [196]:
# Fit our scaler, named 'scaler_2' to our data, which produced a new 
# Standard Scaler object, which we call X_scaler_2
X_scaler_2 = scaler_2.fit(X_train_2)

In [197]:
# Scale the data
X_train_2_scaled = X_scaler_2.transform(X_train_2)
X_test_2_scaled = X_scaler_2.transform(X_test_2)

In [198]:
# Instantiate a linear SVM model
from sklearn.svm import SVC
model_5 = SVC(kernel='linear')

In [199]:
# Fit/train our model using the scaled training data
model_5.fit(X_train_2_scaled, y_train_2)

SVC(kernel='linear')

In [200]:
# Make predictions using the test data
y_pred_2 = model_5.predict(X_test_2_scaled)
results_SVM = pd.DataFrame({
    "Prediction": y_pred_2,
    "Actual": y_test_2
}).reset_index(drop=True)

In [201]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_2, y_pred_2)

0.9311809685641461

In [202]:
balanced_accuracy_score(y_test_2, y_pred_2)

0.5

In [203]:
f1_score(y_test_2, y_pred_2, average="weighted")

0.8979976608414467

In [204]:
cm = confusion_matrix(y_test_2, y_pred_2)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [205]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print("Classification Report")
print(classification_report(y_test_2, y_pred_2))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1096,0
Actual 1,81,0


Classification Report
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1096
           1       0.00      0.00      0.00        81

    accuracy                           0.93      1177
   macro avg       0.47      0.50      0.48      1177
weighted avg       0.87      0.93      0.90      1177



/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

## Attempt 6: Decision Tree

In [206]:
from sklearn import tree

# Instantiate our learning model
model_6 = tree.DecisionTreeClassifier()

In [207]:
# Fit our model to our trained, scaled data
model_6 = model_6.fit(X_train_scaled, y_train)

In [208]:
# Make predictions using the testing data
predictions = model_6.predict(X_test_scaled)

# check your predictions
predictions[:20]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [209]:
# Evaluate our model
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)
acc_score

0.8615123194562447

In [210]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1005,91
Actual 1,72,9


Accuracy Score : 0.8615123194562447
Classification Report
              precision    recall  f1-score   support

           0       0.93      0.92      0.92      1096
           1       0.09      0.11      0.10        81

    accuracy                           0.86      1177
   macro avg       0.51      0.51      0.51      1177
weighted avg       0.88      0.86      0.87      1177



## Attempt 7: Random Forest

In [211]:
from sklearn.ensemble import RandomForestClassifier
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)

# Remember, the accuracy_score function requires 2 arguments, the test and the pred
print(f" Random forest predictive accuracy: {accuracy_score(y_test, y_pred):.3f}")

 Random forest predictive accuracy: 0.931


In [212]:
cm_RandomForest = confusion_matrix(y_test, y_pred)
cm_RandomForest_df = pd.DataFrame(
    cm_RandomForest, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [213]:
# Displaying results
print("Confusion Matrix")
display(cm_RandomForest_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1096,0
Actual 1,81,0


Accuracy Score : 0.8615123194562447
Classification Report
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1096
           1       0.00      0.00      0.00        81

    accuracy                           0.93      1177
   macro avg       0.47      0.50      0.48      1177
weighted avg       0.87      0.93      0.90      1177



/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

# Attempt 8: Gradient Boosting

In [214]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error

In [215]:
# Choose the best learning rate

learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train_scaled, y_train)
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test)))
    print()

Learning rate:  0.05
Accuracy score (training): 0.932
Accuracy score (validation): 0.931

Learning rate:  0.1
Accuracy score (training): 0.932
Accuracy score (validation): 0.931

Learning rate:  0.25
Accuracy score (training): 0.934
Accuracy score (validation): 0.929

Learning rate:  0.5
Accuracy score (training): 0.934
Accuracy score (validation): 0.927

Learning rate:  0.75
Accuracy score (training): 0.935
Accuracy score (validation): 0.919

Learning rate:  1
Accuracy score (training): 0.938
Accuracy score (validation): 0.919



In [216]:
# Looks like a learning rate of 0.25 has the best results. 

classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=0.5,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=0)

# Fit the model
classifier.fit(X_train_scaled, y_train)

# Make Prediction
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).sample(20)

,Prediction,Actual
PARCEL,,
5084029034,0,0
5086023022,0,0
5086011015,0,0
5514003033,0,0
5087019010,0,0
5089017026,0,0
5510017025,0,0
5085028026,0,0
5070008003,0,0


In [217]:
# Evaluate the model

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.9269328802039083


In [218]:
# Generate the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,1091,5
Actual 1,81,0


# Attempt 9: Adaptive Boosting

In [219]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [220]:
# lets use the training data from the reduced dataframe (the one with fewer columns)

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2), n_estimators=200,
    algorithm='SAMME.R', learning_rate=0.5)

# fit our model to our scaled data
ada_clf.fit(X_train_2_scaled, y_train_2)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2),
                   learning_rate=0.5, n_estimators=200)

In [221]:
# Evaluate the model
y_pred_3 = ada_clf.predict(X_test_2_scaled)

In [222]:
# Remember, the accuracy_score function requires 2 arguments, the test and the pred
print(f" Adaptive Boosting predictive accuracy: {accuracy_score(y_test_2, y_pred_3):.3f}")

 Adaptive Boosting predictive accuracy: 0.912


In [223]:
cm_AdaBoost = confusion_matrix(y_test_2, y_pred_3)
cm_AdaBoost_df = pd.DataFrame(
    cm_AdaBoost, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [224]:
# Displaying results
print("Confusion Matrix")
display(cm_AdaBoost_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test_2, y_pred_3))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1072,24
Actual 1,79,2


Accuracy Score : 0.9269328802039083
Classification Report
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      1096
           1       0.08      0.02      0.04        81

    accuracy                           0.91      1177
   macro avg       0.50      0.50      0.50      1177
weighted avg       0.87      0.91      0.89      1177



# Attempt 10: Use K-Folds & Then Logistic Regression

In [251]:
# partition our data into training and holdout sets
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import max_error

X_train_KFolds, X_holdout_KFolds, y_train_KFolds, y_holdout_KFolds = train_test_split(X_2, y_2, stratify=y, random_state=78, shuffle=True)

In [252]:
# from sklearn.model_selection import KFold

# for train_index, test_index in KFold(n_splits=5).split(X_train_KFolds):
#     X_data = X_train_KFolds.iloc[train_index]
#     y_data = y_train_KFolds.iloc[train_index]

In [253]:
max_err = 0

for train_index, test_index in KFold(n_splits = 5).split(X_train_KFolds):

    model = RandomForestRegressor(
        # our hyperparameters, except n_jobs
        n_jobs = -1,
        n_estimators = 300,
        max_depth = 5,
        max_features = 'sqrt'
    )
    
    model.fit(X_train_KFolds.iloc[train_index], y_train_KFolds.iloc[train_index])
    y_pred_KFolds = model.predict(X_train_KFolds.iloc[test_index])
    max_err += max_error(y_pred_KFolds, y_train_KFolds.iloc[test_index])
    
max_err / 5

0.9632954983330453

In [254]:
model.score(X_holdout_KFolds, y_holdout_KFolds)

-0.01350715872067143

In [255]:
print(f" KFolds + Random Forest accuracy: {accuracy_score(y_holdout_KFolds, y_pred_KFolds):.3f}")

ValueError: Found input variables with inconsistent numbers of samples: [1177, 705]

# Attempt 11: Use a Logistic Regression, but use unscaled data

In [149]:
# Split our data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

In [150]:
# Create the learning model

classifier = LogisticRegression(solver='lbfgs',
                                max_iter=300,
                                random_state=78,
                                class_weight="balanced")

In [151]:
# fit our learning model to our scaled training data

classifier.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [152]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test, "PARCEL": y_test.index.values}).reset_index(drop=True)
results.sample(20)

,Prediction,Actual,PARCEL
963,1,0,5088017023
586,1,0,5514004045
97,1,0,5512014025
886,0,0,5525036006
3,1,0,5070007020
512,0,0,5513020017
488,1,0,5511023002
854,1,0,5526025015
735,1,0,5510019025
484,1,0,5510003018


In [153]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.5845369583687341


In [154]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average="weighted")

0.686189422125401

In [155]:
# See our results in a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[654, 442],
       [ 47,  34]])

In [156]:
# evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.60      0.73      1096
           1       0.07      0.42      0.12        81

    accuracy                           0.58      1177
   macro avg       0.50      0.51      0.42      1177
weighted avg       0.87      0.58      0.69      1177



# Attempt 12: PCA + Logistic Regression

In [274]:
# bring in our dataframe

Beverly_Grove_df = pd.read_csv('Beverly_Grove_Cleaned.csv', index_col='PARCEL')
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0


In [295]:
# drop any nulls

Beverly_Grove_df = Beverly_Grove_df.dropna()

In [276]:
Beverly_Grove_df['TAXAMT'] = Beverly_Grove_df['TAXAMT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(int)

In [277]:
# Make sure our dataframe is only SFR's so exclude every property that has an '#' in its site address

Beverly_Grove_df = Beverly_Grove_df[~Beverly_Grove_df['SITEADDRESS'].str.contains('#')]
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0


In [278]:
# Now drop the SiteAddress column
Beverly_Grove_df.drop(['SITEADDRESS'], axis=1, inplace=True)

In [279]:
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0


In [280]:
# Step 1: Create our X and y

y = Beverly_Grove_df['Did it sell?']
X = Beverly_Grove_df.drop(['Did it sell?'], 1)

In [281]:
APN = Beverly_Grove_df.index.values
APN

array([5085010022, 5085033008, 5085032022, ..., 5510018012, 5510017023,
       5514001029])

In [282]:
# Step 2: Use PCA to reduce dimension to three principal components.
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

In [283]:
X_pca

array([[ 1037442.40088322,   222949.06197812,    99008.44211244],
       [ -932464.7141857 ,   251848.62995472,   -90439.60157538],
       [ -393042.03210882,  -290764.40544306,   -27460.47099801],
       ...,
       [ -314557.49024403,  -369458.72763661,   -27020.9155848 ],
       [ 2038277.22062523,   419857.55993186, -1288530.60059949],
       [ -962055.78924987,   281605.81570128,   -93574.2647082 ]])

In [284]:
# Step 3: Train, test, split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, random_state=78, stratify=y)

In [285]:
# Step 4: Scale our data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit our scaler, named 'scaler' to our data, which produces a new StandardScaler object
# which we call 'X_scaler'
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [286]:
X_train_scaled_APN = []

In [293]:
# for index, parcel in enumerate(X_train.index.values):
#      X_train_scaled_APN.append(np.insert(X_train_scaled[index], 0, [parcel]))

In [288]:
# Step 5:Create the learning model

log_classifier = LogisticRegression(solver='lbfgs',
                                max_iter=300,
                                random_state=78,
                                class_weight="balanced")

# Train the model
log_classifier.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test_scaled)

print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.613


In [289]:
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test, "PARCEL": y_test.index.values}).reset_index(drop=True)
results.sample(20)

,Prediction,Actual,PARCEL
53,0,0,5089018042
952,0,0,5088003055
999,0,0,5510017025
1123,0,0,5085009005
602,0,1,5512014022
245,0,0,5526022026
914,0,0,5086027022
394,1,0,5088004044
740,1,0,5510014004
260,0,0,5526018010


In [290]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[693, 403],
       [ 53,  28]])

In [291]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average="weighted")

0.7081876800594455

In [292]:
# evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.63      0.75      1096
           1       0.06      0.35      0.11        81

    accuracy                           0.61      1177
   macro avg       0.50      0.49      0.43      1177
weighted avg       0.87      0.61      0.71      1177



# Attempt 13: PCA + Resampling (SMOTE) + LogisticRegression

In [296]:
# bring in our dataframe

Beverly_Grove_df = pd.read_csv('Beverly_Grove_Cleaned.csv', index_col='PARCEL')
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0


In [297]:
# drop any nulls

Beverly_Grove_df = Beverly_Grove_df.dropna()

In [298]:
# Change the datatype of these two columns into ints

Beverly_Grove_df['TAXAMT'] = Beverly_Grove_df['TAXAMT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(int)

In [299]:
# Make sure our dataframe is only SFR's so exclude every property that has an '#' in its site address

Beverly_Grove_df = Beverly_Grove_df[~Beverly_Grove_df['SITEADDRESS'].str.contains('#')]
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0


In [300]:
# Now drop the SiteAddress column
Beverly_Grove_df.drop(['SITEADDRESS'], axis=1, inplace=True)

In [301]:
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0


In [302]:
# Step 1: Create our X and y

y = Beverly_Grove_df['Did it sell?']
X = Beverly_Grove_df.drop(['Did it sell?'], 1)

In [303]:
APN = Beverly_Grove_df.index.values
APN

array([5085010022, 5085033008, 5085032022, ..., 5510018012, 5510017023,
       5514001029])

In [304]:
# Step 2: Use PCA to reduce dimension to three principal components.
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

In [305]:
X_pca

array([[ 1037442.40088322,   222949.06197813,    99008.44211243],
       [ -932464.7141857 ,   251848.62995472,   -90439.60157539],
       [ -393042.03210882,  -290764.40544306,   -27460.470998  ],
       ...,
       [ -314557.49024403,  -369458.72763661,   -27020.9155848 ],
       [ 2038277.22062523,   419857.55993186, -1288530.60059949],
       [ -962055.78924987,   281605.81570128,   -93574.2647082 ]])

In [306]:
# Step 3: Train, test, split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, random_state=78, stratify=y)

In [307]:
# Step 4: Scale our data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit our scaler, named 'scaler' to our data, which produces a new StandardScaler object
# which we call 'X_scaler'
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [308]:
X_train_scaled_APN = []

In [310]:
# Step 5: Resample our data using the SMOTE method

from imblearn.over_sampling import SMOTE
X_resampled_SMOTE, y_resampled_SMOTE = SMOTE(random_state=1, 
                               sampling_strategy='auto').fit_resample(
                               X_train_scaled, y_train)

In [311]:
Counter(y_resampled_SMOTE)

Counter({0: 3287, 1: 3287})

In [312]:
# Step 6: Instantiate our model and fit it to our data

model_7 = LogisticRegression(solver='lbfgs', random_state=78, max_iter=300, class_weight="balanced")
model_7.fit(X_resampled_SMOTE, y_resampled_SMOTE)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [313]:
# Step 7: Calculated the balanced accuracy score

y_pred = model_7.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.4914841849148418

In [314]:
f1_score(y_test, y_pred, average="weighted")

0.719691707488016

In [315]:
confusion_matrix(y_test, y_pred)

array([[712, 384],
       [ 54,  27]])

In [316]:
# evaluate
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.65      0.76      1096
           1       0.07      0.33      0.11        81

    accuracy                           0.63      1177
   macro avg       0.50      0.49      0.44      1177
weighted avg       0.87      0.63      0.72      1177

